# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Creating or Loading Evaluation Labels

In [ ]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

## Part III(a): Creating Evaluation Labels in the `Viewer`

We repeat our definition of the `Spouse` `Candidate` subclass from Part II.

In [ ]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## Loading the development `CandidateSet`

We will start by viewing the development `CandidateSet` we created in Part II in the `Viewer`.

First we reload the development `CandidateSet`.

In [ ]:
from snorkel.models import CandidateSet

cs = session.query(CandidateSet).filter(CandidateSet.name == 'News Development Candidates').one()
cs

## Labeling the `CandidateSet` in the `Viewer`

We create a `Viewer` to annotate them manually.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(cs, session, annotator_name="Tutorial Part III User")
else:
    sv = None

We now open the Viewer.

You can mark each `Candidate` as true or false. Try it!

These labels are automatically saved in the database backend, and can be accessed using the annotator's name ('Tutorial Part III User') as the AnnotationKey.

In [ ]:
sv

## Part III(b): Loading External Evaluation Labels

In case you want to skip annotating the data yourself, this tutorial also includes labels for the development and test sets.

Snorkel uses `AnnotationManager` objects to manage information associated with each `Candidate`. Types of annotations include `Label`, `Feature`, and `Prediction`. We will use annotations further in parts IV and V.

First, we create an `AnnotationKeySet` and `AnnotationKey` for our labels.

In [ ]:
from snorkel.queries import get_or_create_single_key_set

gold_key_set, gold_key = get_or_create_single_key_set(session, 'News Gold Labels')

Next, we read in triples, two spans represented by their _stable ids_, representing a `Spouse` `Candidate` and a value, where -1 means False and 1 means True.

In [ ]:
import pandas as pd
from utils import add_spouse_label

gold_labels = pd.read_csv('data/gold_labels.tsv', sep="\t")
for index, row in gold_labels.iterrows():
    add_spouse_label(session, gold_key, Spouse, row['person1'], row['person2'], row['label'])

Next, in Part IV, we will build a model to predict these labels using data programming.